In [4]:
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
from scipy.interpolate.interpolate import interp1d
import plotly.graph_objects as go

In [5]:
class PropellerComparison:
    def __init__(self, chart, blades):
        self.chart = chart
        self.blades = blades
        self.main = self.load_data(blades=3 if chart=='cp' else 4, large_angles=True)
        self.other = self.load_data(blades=blades)
        self.last = self.load_data(blades=4 if self.blades==2 else 2)
        self.ratio = self.calculate_ratio()
        self.normalized = self.normalize_ratio()
    
    def melt(self, df):
        df = df.melt(id_vars = ['x'], var_name = 'y', value_name = 'z').dropna()
        df.y = df.y.astype(float)
        df.x = df.x.round(3)
        return df[df.x >= 0]
    
    def load_data(self, blades, large_angles=False):
        path = r'data\large_angles\clark_{}_{}.csv' if large_angles else r'data\clark_{}_{}.csv'
        data = pd.read_csv(path.format(blades, self.chart))
        if self.chart == 'eff' and not large_angles:
            data = pd.DataFrame(0, index=[0], columns=data.columns).append(data, ignore_index=True)
        data = self.melt(data)
        return data[data.z >= 0].reset_index(drop=True)
    
    def calculate_ratio(self):
        ratio = pd.merge(self.main, self.other, how='inner', on=['x', 'y'], suffixes=('_main', '_other'))
        ratio['z'] = ratio['z_other'] / ratio['z_main']
        ratio = ratio.replace([np.inf, -np.inf], np.nan).dropna()
        ratio = ratio[ratio.z.between(0.5, 5)].reset_index(drop=True)
        return ratio[['x', 'y', 'z']]
        
    def normalize_ratio(self):
        normalized = self.ratio[self.ratio.y > 15].copy()
        for y in normalized.y.unique():
            seriesEnd = self.main[self.main.y == y].max().x
            normalized.loc[normalized.y == y, 'x'] /= seriesEnd
        cutoff = 0.97 if self.chart == 'cp' else 0.995
        return normalized[normalized.x < cutoff]
        
    def fit_polynomial(self, append=False, degree=9):
        ratio = pd.DataFrame()
        z = np.poly1d(np.polyfit(self.normalized.x, self.normalized.z, degree))
        for y in [50, 55, 60]:
            seriesEnd = self.main[self.main.y == y].max().x
            x_vals = np.arange(0, self.normalized.x.max(), 0.001 / seriesEnd)
            ratio = ratio.append(pd.DataFrame({'x': np.round(x_vals * seriesEnd, 3), 
                                               'y': y, 
                                               'z': z(x_vals)
                                              }), ignore_index=True)
        self.ratio = self.ratio.append(ratio, ignore_index=True) if append else ratio
    
    def calculate_data(self):
        data = pd.merge(self.main, self.ratio, how='inner', on=['x', 'y'], suffixes=('_main', '_ratio'))
        data = data[data.y > 45]
        data['z'] = data['z_main'] * data['z_ratio']
        data = data[['x', 'y', 'z']]
        self.other = self.other.append(data, ignore_index=True)
        
    def normalized_view(self):
        fig = go.Figure(layout=go.Layout(margin=dict(l=0, r=0, b=0, t=0), width=840, height=360))
        fig.add_trace(go.Scatter(x=self.normalized.x, y=self.normalized.z, mode='markers', marker=dict(size=3)))
        fig.show()
        
    def multifit(self, degrees=[3, 4, 9]):
        fits = pd.DataFrame()
        fig = go.Figure(layout=go.Layout(margin=dict(l=0, b=0, t=0), width=1120, height=480))
        points = self.normalized
        opacity = 0.25 if self.chart=='cp' else 0.05
        fig.add_trace(go.Scatter(x=points.x, y=points.z, mode='markers', 
                                 marker=dict(size=3, opacity=opacity), name='ratios'))
        x_vals = np.linspace(0, self.normalized.x.max(), 100)
        for i in degrees:
            p, ssr, rank, sv, ct = np.polyfit(points.x, points.z, i, full=True)
            y = np.poly1d(p)  
            fig.add_trace(go.Scatter(x=x_vals, y=y(x_vals), name=i))
            fits = fits.append({'Degree': i, 'SSR': ssr[0]}, ignore_index=True)
        fig.show()
        return fits.round(3).T
        
    def save_data(self):
        path = r'data\large_angles\clark_{}_{}.csv'
        main = self.main.pivot(index='x', columns='y', values='z')
        other = self.other.drop_duplicates(subset=['x', 'y'])
        other = other.pivot(index='x', columns='y', values='z')
        main.to_csv(path.format(3 if self.chart=='cp' else 4, self.chart))
        other.to_csv(path.format(self.blades, self.chart))
            

In [6]:
def draw(obj, three=False):
    fig = go.Figure(layout=layout3d(obj))
    fig.add_trace(trace(obj.other, 'red', comparison.blades))
    fig.add_trace(trace(obj.main, 'black', 3 if obj.chart=='cp' else 4))
    if three:
        fig.add_trace(trace(obj.last, 'green', 2 if comparison.blades==4 else 4))
    elif obj.other.y.max() < 60:
        fig.add_trace(trace(obj.ratio, 'blue', 'ratio', 2))
    fig.show()
    
def trace(df, color, name, size=3):
    return go.Scatter3d(
        x=df.x, y=df.y, z=df.z,
        mode='markers',
        marker=dict(size=size, color=color, opacity=0.9),
        name=name)
    
def layout3d(obj):
    camera = dict(eye=dict(x=1.3, y=-1.3, z=0.5),
                  center=dict(y=-0.1, z=-0.15))
    scene = dict(
        xaxis = dict(title='J'),
        yaxis = dict(title='Angle'),
        zaxis = dict(title='Eff' if obj.chart=='eff' else 'Cp')
    )
    legend = dict(font = {'size': 12}, itemsizing = 'trace')
    return go.Layout(margin=dict(l=0, r=0, b=0, t=0), 
                     width=1024, height=768, 
                     scene=scene,
                     scene_camera=camera,
                     legend_title_text='Data',
                     legend=legend)

def layout(obj):
    return go.Layout(margin=dict(l=0, r=0, b=0, t=0))

In [9]:
comparison = PropellerComparison(blades=3, chart='eff')
comparison.fit_polynomial(degree=9, append=True)
comparison.calculate_data()
draw(comparison, three=False)
comparison.save_data()

In [428]:
comparison.multifit()

,0,1,2
Degree,3.000,4.000,9.000
SSR,2.004,1.796,1.515


In [380]:
np.linspace(1, 2, 4) * 2

array([2.        , 2.66666667, 3.33333333, 4.        ])

In [35]:
comparison.other[comparison.other.duplicated(subset=['x', 'y'])]

,x,y,z


In [8]:
comparison.main.pivot(index='x', columns='y', values='z')

y,15.0,20.0,25.0,30.0,35.0,40.0,45.0,50.0,55.0,60.0
x,,,,,,,,,,
0.000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000
0.001,0.00233,0.00186,0.00118,0.00083,0.00068,0.00057,0.00046,0.0004,0.00035,0.00027
0.002,0.00466,0.00372,0.00236,0.00165,0.00135,0.00115,0.00092,0.0008,0.00070,0.00054
0.003,0.00699,0.00559,0.00353,0.00248,0.00203,0.00172,0.00139,0.0012,0.00104,0.00081
0.004,0.00932,0.00745,0.00471,0.00331,0.00271,0.00230,0.00185,0.0016,0.00139,0.00109
...,...,...,...,...,...,...,...,...,...,...
4.530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00722
4.531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00547
4.532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00371
